In [29]:
import os
import json
from typing import List, TypedDict, NotRequired, Any
from datetime import datetime

import pandas as pd
import numpy as np
from numpy import floating


In [30]:
# load the data
applications = pd.read_csv(os.path.join(os.getcwd(), 'data.csv'), dtype={"id": int})
applications['application_date'] = pd.to_datetime(applications['application_date'], format='mixed')


In [31]:
applications.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   id                1000 non-null   int64              
 1   application_date  1000 non-null   datetime64[ns, UTC]
 2   contracts         495 non-null    object             
dtypes: datetime64[ns, UTC](1), int64(1), object(1)
memory usage: 23.6+ KB


In [32]:
applications.head(10)

,id,application_date,contracts
0,2925210,2024-02-12 19:22:46.652000+00:00,NaN
1,2925211,2024-02-12 19:24:29.135000+00:00,"[{""contract_id"": 522530, ""bank"": ""003"", ""summa..."
2,2925212,2024-02-12 19:24:41.493000+00:00,NaN
3,2925213,2024-02-12 19:24:29.135000+00:00,"[{""contract_id"": 522530, ""bank"": ""003"", ""summa..."
4,2925214,2024-02-12 19:24:56.857000+00:00,NaN
5,2925215,2024-02-12 19:25:10.176000+00:00,"[{""contract_id"": 522530, ""bank"": ""003"", ""summa..."
6,2925216,2024-02-12 19:25:10.176000+00:00,"[{""contract_id"": 522530, ""bank"": ""003"", ""summa..."
7,2925217,2024-02-12 19:39:43.431000+00:00,NaN
8,2925218,2024-02-12 19:40:15.507000+00:00,"[{""contract_id"": """", ""bank"": ""062"", ""summa"": ""..."
9,2925219,2024-02-12 19:40:21.729000+00:00,NaN


In [33]:
pd.DataFrame(json.loads(applications['contracts'][991]))


,contract_id,bank,summa,loan_summa,claim_date,claim_id,contract_date
0,18410,004,49164700,0,03.04.2019,18410,03.04.2019
1,,004,,,22.02.2021,43136,
2,,062,,,14.02.2022,3304534,
3,3304674,062,405000000,217320883,14.02.2022,3304674,14.02.2022
4,,004,,,18.02.2022,62750,
5,,014,,,31.05.2023,F00895453,


In [34]:
applications['contracts'][991]

'[{"contract_id": 18410, "bank": "004", "summa": 49164700, "loan_summa": 0, "claim_date": "03.04.2019", "claim_id": 18410, "contract_date": "03.04.2019"}, {"contract_id": "", "bank": "004", "summa": "", "loan_summa": "", "claim_date": "22.02.2021", "claim_id": 43136, "contract_date": ""}, {"contract_id": "", "bank": "062", "summa": "", "loan_summa": "", "claim_date": "14.02.2022", "claim_id": 3304534, "contract_date": ""}, {"contract_id": 3304674, "bank": "062", "summa": 405000000, "loan_summa": 217320883, "claim_date": "14.02.2022", "claim_id": 3304674, "contract_date": "14.02.2022"}, {"contract_id": "", "bank": "004", "summa": "", "loan_summa": "", "claim_date": "18.02.2022", "claim_id": 62750, "contract_date": ""}, {"contract_id": "", "bank": "014", "summa": "", "loan_summa": "", "claim_date": "31.05.2023", "claim_id": "F00895453", "contract_date": ""}]'

In [35]:
# filter out rows with null contacts
not_null_contacts = ~pd.isna(applications['contracts'])


In [36]:
# convert not null contacts to list of contracts

applications.loc[not_null_contacts, 'contracts'] = applications.loc[not_null_contacts, 'contracts'].apply(
    lambda x: json.loads(x) if isinstance(json.loads(x), list) else [json.loads(x)]
)

In [37]:
# all types used in code 
class Contract(TypedDict):
    contract_id: int | str
    bank: NotRequired[str]
    summa: int | str
    loan_summa: int | str
    claim_date: str
    claim_id: int | str
    contract_date: str

Contracts = List[Contract]

In [38]:
# adding new columns to store features
applications['tot_claim_cnt_l180d'] = -3
applications['disb_bank_loan_wo_tbc'] = -3
applications['day_sinlastloan'] = -3
applications['disb_bank_loan_by_tbc'] = -3

applications['loan_to_claim_ratio'] = 0.0

applications['average_summa'] = -1.0
applications['average_loan_summa'] = -1.0


In [39]:
applications.head(10)

,id,application_date,contracts,tot_claim_cnt_l180d,disb_bank_loan_wo_tbc,day_sinlastloan,disb_bank_loan_by_tbc,loan_to_claim_ratio,average_summa,average_loan_summa
0,2925210,2024-02-12 19:22:46.652000+00:00,NaN,-3,-3,-3,-3,0.0,-1.0,-1.0
1,2925211,2024-02-12 19:24:29.135000+00:00,"[{'contract_id': 522530, 'bank': '003', 'summa...",-3,-3,-3,-3,0.0,-1.0,-1.0
2,2925212,2024-02-12 19:24:41.493000+00:00,NaN,-3,-3,-3,-3,0.0,-1.0,-1.0
3,2925213,2024-02-12 19:24:29.135000+00:00,"[{'contract_id': 522530, 'bank': '003', 'summa...",-3,-3,-3,-3,0.0,-1.0,-1.0
4,2925214,2024-02-12 19:24:56.857000+00:00,NaN,-3,-3,-3,-3,0.0,-1.0,-1.0
5,2925215,2024-02-12 19:25:10.176000+00:00,"[{'contract_id': 522530, 'bank': '003', 'summa...",-3,-3,-3,-3,0.0,-1.0,-1.0
6,2925216,2024-02-12 19:25:10.176000+00:00,"[{'contract_id': 522530, 'bank': '003', 'summa...",-3,-3,-3,-3,0.0,-1.0,-1.0
7,2925217,2024-02-12 19:39:43.431000+00:00,NaN,-3,-3,-3,-3,0.0,-1.0,-1.0
8,2925218,2024-02-12 19:40:15.507000+00:00,"[{'contract_id': '', 'bank': '062', 'summa': '...",-3,-3,-3,-3,0.0,-1.0,-1.0
9,2925219,2024-02-12 19:40:21.729000+00:00,NaN,-3,-3,-3,-3,0.0,-1.0,-1.0


In [40]:
def tot_claim_cnt_l180d(claims: Contracts) -> int:
    today = datetime.now()
    return len(list(filter(lambda claim: (today - datetime.strptime(claim['claim_date'], '%d.%m.%Y')).days < 180, claims)))

In [41]:
def disb_bank_loan_wo_tbc(claims: Contracts) -> int:
    filtered_claims = list(
        filter(lambda x: x.get('bank', None) not in ['LIZ', 'LOM', 'MKO', 'SUG', None] and x['contract_date'],
               claims)
    )
    if len(filtered_claims) == 0:
        return -1

    return sum([int(claim['loan_summa']) for claim in filtered_claims if str(claim['loan_summa']).isdigit()])

In [42]:
def day_sinlastloan(claims: List[Contract], application_date: datetime) -> int:

    claims = list(filter(lambda x: x['summa'] != '', claims))
    if not claims:
        return -1

    sorted_claims = sorted(claims, key=lambda x: datetime.strptime(x['claim_date'], '%d.%m.%Y'), reverse=True)
    return (application_date.date() - datetime.strptime(sorted_claims[0]['claim_date'], '%d.%m.%Y').date()).days


In [43]:
def disb_bank_loan_by_tbc(claims: Contracts) -> int:
    filtered_claims = list(
        filter(lambda x: x.get('bank', None) in ['LIZ', 'LOM', 'MKO', 'SUG'] and x['contract_date'],
               claims)
    )
    if len(filtered_claims) == 0:
        return -1

    return sum([int(claim['loan_summa']) for claim in filtered_claims if str(claim['loan_summa']).isdigit()])

In [44]:
def loan_to_claim_ratio(claims: Contracts) -> float:
    claims = list(filter(lambda x: x['summa'] != '', claims))
    if not claims:
        return -1

    return sum([int(claim['loan_summa']) for claim in claims if str(claim['loan_summa']).isdigit()]) / sum(
        [int(claim['summa']) for claim in claims if str(claim['summa']).isdigit()])

In [45]:
def average_summa(claims: Contracts) -> floating[Any]:
    claims = list(filter(lambda x: x['summa'] != '', claims))
    if not claims:
        return -1

    return np.mean([int(claim['summa']) for claim in claims if str(claim['summa']).isdigit()])

In [46]:
def average_loan_summa(claims: Contracts) -> floating[Any]:
    claims = list(filter(lambda x: x['loan_summa'] != '', claims))
    if not claims:
        return -1

    return np.mean([int(claim['loan_summa']) for claim in claims if str(claim['loan_summa']).isdigit()])

In [47]:

applications.loc[not_null_contacts, 'tot_claim_cnt_l180d'] = applications.loc[not_null_contacts]['contracts'].apply(tot_claim_cnt_l180d)

In [48]:

applications.loc[not_null_contacts, 'disb_bank_loan_wo_tbc'] = applications.loc[not_null_contacts]['contracts'].apply(disb_bank_loan_wo_tbc)


In [49]:

applications.loc[not_null_contacts, 'day_sinlastloan'] = applications.loc[not_null_contacts].apply(lambda columns: day_sinlastloan( columns.contracts, columns.application_date), axis=1)

In [50]:

applications.loc[not_null_contacts, 'disb_bank_loan_by_tbc'] = applications.loc[not_null_contacts]['contracts'].apply(disb_bank_loan_by_tbc)

In [51]:
applications.loc[not_null_contacts, 'loan_to_claim_ratio'] = applications.loc[not_null_contacts]['contracts'].apply(loan_to_claim_ratio)

In [52]:
applications.loc[not_null_contacts, 'average_summa'] = applications.loc[not_null_contacts]['contracts'].apply(average_summa)

In [53]:
applications.loc[not_null_contacts, 'average_loan_summa'] = applications.loc[not_null_contacts]['contracts'].apply(average_loan_summa)

In [54]:
applications

,id,application_date,contracts,tot_claim_cnt_l180d,disb_bank_loan_wo_tbc,day_sinlastloan,disb_bank_loan_by_tbc,loan_to_claim_ratio,average_summa,average_loan_summa
0,2925210,2024-02-12 19:22:46.652000+00:00,NaN,-3,-3,-3,-3,0.000000,-1.000000e+00,-1.0
1,2925211,2024-02-12 19:24:29.135000+00:00,"[{'contract_id': 522530, 'bank': '003', 'summa...",22,0,430,-1,0.000000,5.033333e+08,0.0
2,2925212,2024-02-12 19:24:41.493000+00:00,NaN,-3,-3,-3,-3,0.000000,-1.000000e+00,-1.0
3,2925213,2024-02-12 19:24:29.135000+00:00,"[{'contract_id': 522530, 'bank': '003', 'summa...",22,0,430,-1,0.000000,5.033333e+08,0.0
4,2925214,2024-02-12 19:24:56.857000+00:00,NaN,-3,-3,-3,-3,0.000000,-1.000000e+00,-1.0
...,...,...,...,...,...,...,...,...,...,...
995,2926205,2024-02-13 06:09:54.210000+00:00,"[{'contract_id': 18410, 'bank': '004', 'summa'...",0,217320883,729,-1,0.478507,2.270824e+08,108660441.5
996,2926206,2024-02-13 06:09:54.306000+00:00,NaN,-3,-3,-3,-3,0.000000,-1.000000e+00,-1.0
997,2926207,2024-02-13 06:09:55.661000+00:00,NaN,-3,-3,-3,-3,0.000000,-1.000000e+00,-1.0
998,2926208,2024-02-13 06:09:57.024000+00:00,NaN,-3,-3,-3,-3,0.000000,-1.000000e+00,-1.0


In [55]:
applications.to_csv('contract_features.csv', index=False)